## Cree, entrene e implemente un modelo de machine learning con Amazon SageMaker

### Introducción
En este tutorial, aprenderá a utilizar Amazon SageMaker para crear, entrenar e implementar un modelo de machine learning (ML) por medio del algoritmo de ML XGBoost. Amazon SageMaker es un servicio completamente administrado que brinda a todos los científicos de datos y desarrolladores la capacidad de crear, entrenar e implementar modelos de machine learning rápidamente.
En general, trasladar los modelos de machine learning desde la etapa de conceptualización hasta la etapa de producción es sumamente complejo e implica mucho tiempo. Debe administrar grandes cantidades de datos para entrenar el modelo, elegir el mejor algoritmo posible para entrenarlo, administrar la capacidad de cómputo mientras se lo entrena y, luego, implementar el modelo en un entorno de producción. Amazon SageMaker reduce esta complejidad y facilita de manera significativa la creación y la implementación de modelos de machine learning. Una vez que elige los algoritmos y los marcos adecuados de la amplia gama de opciones disponibles, SageMaker administra toda la infraestructura subyacente para entrenar su modelo a escala de petabytes e implementarlo en la producción.

En este tutorial, asumirá el rol de un desarrollador de machine learning que trabaja en un banco. Se le solicita que desarrolle un modelo de machine learning para predecir si los clientes se inscribirán para un certificado de depósito.
En este tutorial, aprenderá a hacer lo siguiente:

1. Creará una instancia de cuaderno de SageMaker
2. Preparará los datos
3. Entrenará el modelo para aprender de los datos
4. Implementará el modelo
5. Evaluará el rendimiento de su modelo de ML

El modelo se entrenará con el [Conjunto de datos de marketing bancario](https://archive.ics.uci.edu/ml/datasets/bank+marketing) que contiene información demográfica de los clientes, respuestas a los eventos de marketing y factores externos. Los datos se etiquetaron para su conveniencia. Una columna en el conjunto de datos identifica si el cliente está inscrito para algún producto que ofrece el banco. Una versión de este conjunto de datos está a disposición del público en el Machine Learning Repository de la Universidad de California, Irvine.

In [3]:
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.serializers import CSVSerializer

In [4]:
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
xgboost_container = sagemaker.image_uris.retrieve("xgboost", my_region, "latest")

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + xgboost_container + " container for your SageMaker endpoint.")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Success - the MySageMakerInstance is in the us-west-2 region. You will use the 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


Cree el bucket S3 para almacenar sus datos. Copie y pegue el siguiente código en una nueva celda de código y seleccione Ejecutar.

Nota: Asegúrese de sustituir el bucket_name por un nombre de bucket S3 único.

In [5]:
bucket_name = 'sagemaker-test-bucketml'
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [6]:
try:
  urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
  print('Success: downloaded bank_clean.csv.')
except Exception as e:
  print('Data load error: ',e)

try:
  model_data = pd.read_csv('./bank_clean.csv',index_col=0)
  print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


### Mezcle y divida los datos en datos de entrenamiento y datos de prueba.

Los datos de entrenamiento (el 70 % de los clientes) se utilizarán durante el ciclo de entrenamiento del modelo. Debe utilizar la optimización basada en gradientes para refinar de forma iterativa los parámetros del modelo. La optimización basada en gradientes es una forma de encontrar valores de parámetros del modelo que minimicen sus errores, mediante el uso de gradientes de la función de pérdida del modelo.

Los datos de prueba (el 30 % restante de los clientes) se utilizan para evaluar el rendimiento del modelo y para medir el nivel de generalización de los datos nuevos del modelo entrenado.

In [7]:
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [8]:
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

### Implemente el modelo

In [10]:
sess = sagemaker.Session()
xgb = sagemaker.estimator.Estimator(xgboost_container,role, instance_count=1, instance_type='ml.m4.xlarge',output_path='s3://{}/{}/output'.format(bucket_name, prefix),sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,eta=0.2,gamma=4,min_child_weight=6,subsample=0.8,silent=0,objective='binary:logistic',num_round=100)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [11]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: xgboost-2023-09-14-17-39-51-717


2023-09-14 17:39:51 Starting - Starting the training job...
2023-09-14 17:40:18 Starting - Preparing the instances for training.........
2023-09-14 17:41:48 Downloading - Downloading input data...
2023-09-14 17:42:18 Training - Downloading the training image......
2023-09-14 17:43:19 Uploading - Uploading generated training modelArguments: train
[2023-09-14:17:43:08:INFO] Running standalone xgboost training.
[2023-09-14:17:43:08:INFO] Path /opt/ml/input/data/validation does not exist!
[2023-09-14:17:43:08:INFO] File size need to be processed in the node: 3.38mb. Available memory size in the node: 8543.23mb
[2023-09-14:17:43:08:INFO] Determined delimiter of CSV input is ','
[17:43:08] S3DistributionType set as FullyReplicated
[17:43:08] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[17:43:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 14 pruned nodes, max_depth=5
[0]#011train-error:0.100482
[

Este código implementa el modelo en un servidor y crea un punto de conexión de SageMaker al que puede acceder. Este paso puede tardar varios minutos en completars

In [12]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: xgboost-2023-09-14-17-44-47-741
INFO:sagemaker:Creating endpoint-config with name xgboost-2023-09-14-17-44-47-741
INFO:sagemaker:Creating endpoint with name xgboost-2023-09-14-17-44-47-741


------!

### Predicción

Para predecir si los clientes de los datos de prueba se inscribieron o no en el producto del banco seleccione Ejecutar

In [13]:
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
xgb_predictor.serializer = CSVSerializer() # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


### Evalúe el rendimiento del modelo

Este código compara los valores reales y previstos en una tabla denominada matriz de confusión.

In [14]:
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.5%

Predicted      No Purchase    Purchase
Observed
No Purchase    90% (10769)    37% (167)
Purchase        10% (1133)     63% (288) 



En función de las predicciones, podemos concluir que usted predijo que un cliente se inscribiría para un certificado de depósito exactamente para el 90 % de los clientes en los datos de prueba, con una precisión del 63 % (288/455) para los inscritos y del 90 % (10 769/11 902) para los no inscritos.